<a href="https://colab.research.google.com/github/muhammadghazyy/mask_detector/blob/main/model_building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>